In [ ]:
import urllib.request
import json
import pandas as pd
import calendar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import animation, rc
from tqdm import tqdm

In [ ]:
json_url = urllib.request.urlopen('https://climatereanalyzer.org/clim/sst_daily/json/oisst2.1_natlan1_sst_day.json')
json_data = json.load(json_url)

In [ ]:
def dataframe_from_json(json_data):
  df = pd.DataFrame({'Year': pd.Series(dtype='int'), 'Month': pd.Series(dtype='int'), 'Mean':pd.Series(dtype='float')})
  for i in range(0,43):
    year = int(json_data[i]['name'])
    day_value_in_year = 0
    for month in range(1,13):
      num_days_in_month = calendar.monthrange(year,month)[1]
      numerator = 0
      denominator = 0
      for day in range(1,num_days_in_month+1):
        day_value = json_data[i]['data'][day+day_value_in_year - 1]
        if day_value is not None:
          numerator += day_value
          denominator += 1
      if(denominator != 0):
        df = pd.concat([df, pd.DataFrame([{'Year': year, 'Month': month, 'Mean': numerator/denominator}])], ignore_index=True)
      else:
        df = pd.concat([df, pd.DataFrame([{'Year': year, 'Month': month, 'Mean': 0}])], ignore_index=True)
      day_value_in_year += num_days_in_month
  return df

In [ ]:
df = dataframe_from_json(json_data)
df = df.drop(df[df['Year'] == 1981].index)
df = df.drop(df[df['Mean'] == 0.0].index)
df['Month_str'] = df['Month'].map(lambda x: calendar.month_name[x][0:3])
df['Angle'] = df['Month'].apply(lambda x: (x-1)/12*2*np.pi)
df['Variations'] = 0
for i in range(1,13):
  df['Variations'][df['Month'] == i] = df[df['Month'] == i]['Mean'] - df[df['Month'] == i]['Mean'].mean()

In [ ]:
points = np.array([df.Angle, df.Variations]).T.reshape(-1,1,2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
labels = list(df.Month_str.unique())

In [ ]:
df.reset_index(inplace=True)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(16, 16))

# Set a title for the plot
ax.set_title("North Atlantic Sea Surface Temperature Change (1982-2023) \n", color='white', fontdict={'fontsize': 20})

# Make the plot clockwise
ax.set_theta_direction(-1)

# With the zero (Jan) on top
ax.set_theta_zero_location('N')

ax.set_ylim(-1, 1.4)
ax.set_xticks(np.arange(12)/12*2*np.pi)
ax.set_xticklabels(labels, color='white', fontsize=18)

# Set the year label placeholder
label = ax.text(0, -1, df.Year[0],
            ha='center', va='center', size=24, color='white')

# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df.Variations.min(), df.Variations.max())
lines = []
lc = LineCollection(lines, cmap=plt.cm.coolwarm, norm=norm)
line = ax.add_collection(lc)

# Generate the color bar
cb = fig.colorbar(line, ax=ax)
cb.set_label('Temperature Anomaly', color='white', size=18)
cb.ax.yaxis.set_tick_params(color='white')
cb.outline.set_edgecolor('white')
plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color=r'white', size=18)

# Make the background black
fig.patch.set_facecolor('black')
ax.patch.set_facecolor('black')

# The number of data points we have
Nmonths = segments.shape[0]

bar = tqdm(total=Nmonths)

# Define the function that will generate each frame
def animate(count):
    bar.update()
    lc.set_segments(segments[:count])

    # Set the values used for colormapping
    lc.set_array(df.Variations.iloc[:count])
    lc.set_linewidth(2)

    # Update the year label
    label.set_text(df.Year[count])

    return (lc, label)

# Create the animation object
anim = animation.FuncAnimation(fig, animate,
                               frames=np.arange(1, Nmonths, 1),
                               interval=1, blit=False, repeat=False)

# One second per year
writer = animation.writers['ffmpeg'](fps=12)

# Save the file
anim.save('North Atlantic Sea Surface Temperature Change.mp4', writer=writer)